In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import warnings

In [2]:
# Obtener el directorio actual
path = os.getcwd()

# Concatenar el directorio actual con la ruta al directorio de los datos
path_data = os.path.join(path, "raw")
os.listdir(path_data)

['AVAMET', 'CLEAN_DATA', 'GVA', 'IVIA']

In [3]:
path_save = os.path.join(path_data, "CLEAN_DATA")

In [4]:
name_cols = ['Fecha', 'Hora', 'Velocidad', 'Direccion', 'Temperatura', 'Humedad']

Cada carpeta hace referencia a las diferentes fuentes de los datos climáticos. Puesto que cada formato tiene sus peculiaridades, vamos a limpiar los datos para obtener nuestros datos de forma única.

# 1. CLEANING GVA DATA

In [5]:
path_gva = os.path.join(path_data, "GVA")
os.listdir(path_gva)

['data_Alzira.txt', 'data_Artana.txt', 'data_Beniganim.txt', 'data_Cirat.txt']

In [6]:
vars = ['FECHA', 'HORA', 'Veloc.', 'Direc.', 'Temp.', 'H.Rel.']

fechas = {
    'Alzira': ('2016-06-16', '2016-06-18'),
    'Artana': ('2016-07-25', '2016-07-27'),
    'Beniganim': ('2018-08-06', '2018-08-08'),
    'Cirat': ('2015-06-07', '2015-06-10')
}

In [7]:
for file, key in zip(os.listdir(path_gva), fechas.keys()):
    print(file)
    # Leemos el fichero y seleccionamos las variables de interés
    df = pd.read_csv(os.path.join(path_gva, file), sep="\t", encoding='ISO-8859-1')
    df = df[vars]
    df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d/%m/%Y')

    # Fijamos el intervalo de interés
    inicio, fin = fechas[key]
    df = df[(df['FECHA'] >= inicio) & (df['FECHA'] <= fin)]
    

    # Cambiamos el formato de las fechas y variables.
    df['HORA'] = df['HORA'].astype('int')
    try:
        df['Veloc.'] = df['Veloc.'].astype('float')
        df['Temp.'] = df['Temp.'].astype('float')
    except:
        df['Veloc.'] = df['Veloc.'].str.replace(",", ".").astype('float')
        df['Temp.'] = df['Temp.'].str.replace(",", ".").astype('float')
    df['Direc.'] = df['Direc.'].astype('float')
    df['H.Rel.'] = df['H.Rel.'].astype('float') 
    df['Veloc.'] = df['Veloc.'] * 3.6 #  m/s -> km/h

    # Cambiamos el nombre de las variables
    df.columns = name_cols

    # Guardamos el fichero
    name = file.split(".")[0]
    df.to_csv(os.path.join(path_save, name), sep = ";", decimal = ".", index = False)

data_Alzira.txt
data_Artana.txt
data_Beniganim.txt
data_Cirat.txt


In [8]:
df = pd.read_csv(os.path.join(path_save, "data_Alzira"), sep = ";", decimal = ".")

# 2. Cleaning AVAMET data

In [9]:
path_avamet = os.path.join(path_data, "AVAMET")
os.listdir(path_avamet)

['data_Beneixama.csv', 'data_Beniarda.csv', 'data_VallGallinera.csv']

In [10]:
name_cols = ['Fecha', 'Hora', 'Direccion', 'Velocidad', 'Temperatura', 'Humedad']
vars = ['Fecha', 'Hora', 'vent gra_mit', 'vent vel_mit', "temp mit_mit", "hrel mit_mit"]
fechas = {
    'Beneixama': ('2019-07-15', '2019-07-15'),
    'Beniarda': ('2020-08-28', '2020-08-29'),
    'VallGallinera': ('2020-08-08', '2020-08-10')
}

In [11]:
for file, key in zip(os.listdir(path_avamet), fechas.keys()):
    print(file)
    # Leemos el fichero. Seleccionamos y creamos las variables de interés
    df = pd.read_csv(os.path.join(path_avamet, file))
    df[['Fecha', 'Hora']] = df['data ini'].str.split(" ", expand = True)
    df['Fecha'] = pd.to_datetime(df['Fecha'], format='%m/%d/%y')
    df = df[vars]

    # Fijamos el intervalo de interés
    inicio, fin = fechas[key]
    df = df[(df['Fecha'] >= inicio) & (df['Fecha'] <= fin)]

    # Cambiamos el formato de las fechas y variables.
    try:
        df['vent vel_mit'] = df['vent vel_mit'].astype('float')
        df['temp mit_mit'] = df['temp mit_mit'].astype('float')
        df['hrel mit_mit'] = df['hrel mit_mit'].astype('float')
    except:
        df['vent vel_mit'] = df['vent vel_mit'].astype('str').str.replace(",", ".").astype('float')
        df['temp mit_mit'] = df['temp mit_mit'].astype('str').str.replace(",", ".").astype('float')
        df['hrel mit_mit'] = df['hrel mit_mit'].astype('str').str.replace(",", ".").astype('float')
    
    # Cambiamos el nombre de las variables y guradamos el fichero.
    df.columns = name_cols
    name = file.split(".")[0]
    df.to_csv(os.path.join(path_save, name), sep = ";", decimal = ".", index = False)

data_Beneixama.csv
data_Beniarda.csv
data_VallGallinera.csv
